# 벡터 디비에 저장하기

In [ ]:
# !pip install langchain_community langchain langchain-upstage docx2txt chromadb langchain-chroma

In [6]:
# 워드 파일 불러오기
from langchain_community.document_loaders import Docx2txtLoader
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma  #← Chroma 가져오기

loader = Docx2txtLoader('소득세법(법률)(제19933호)(20240701).docx')
documents = loader.load()

In [10]:
# 문서 쪼개기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
split_documents = text_splitter.split_documents(documents)

In [ ]:
from langchain_upstage import UpstageEmbeddings
embeddings = UpstageEmbeddings( #← OpenAIEmbeddings를 초기화
    model="embedding-query") #← 모델명을 지정

In [21]:
# 벡터 디비(Chroma)에 저장하기
# collection_name : tax
# 벡터를 저장할 경로 : ./vectorDB
embeddings = UpstageEmbeddings(model = 'embedding-query',
                               api_key = 'up_NZqIrRnR6XRpI93EZKtO3UlJhtMWI')
vectorstore = Chroma.from_documents(documents=split_documents,
                                    embedding=embeddings,
                                    collection_name='tax', # 저장할 documents의 별명
                                    persist_directory='./vectorDB') # 저장할 경로

In [22]:
vectorstore_docs = vectorstore.similarity_search('종합소득세 공제 내역', k = 4)

# LLM에게 답변 요청하기

In [26]:
llm = ChatUpstage(model = 'solar-pro', api_key = 'up_NZqIrRnR6XRpI93EZKtO3UlJhtMWI')

In [27]:
query = '종합소득세 공제 내역'
prompt = f"""
아래 [Context]를 참고해서 [query]에 답변해주세요.

[Context]
{vectorstore_docs}

[query]
{query}
"""

In [28]:
ai_response = llm.invoke(prompt)
ai_response

AIMessage(content='종합소득세 공제 내역은 다음과 같습니다:\n\n1. 기본공제: 거주자 본인, 배우자, 부양가족에 대해 1명당 연 150만원 공제\n\n2. 추가공제:\n\t* 경로우대자: 1명당 연 100만원\n\t* 장애인: 1명당 연 200만원\n\t* 6세 이상 20세 이하 또는 65세 이상인 부양가족\n\t* 경로우대자와 장애인에 해당하지 않는 70세 이상인 부양가족\n\t* 자녀(출생, 입양, 6개월 이상 양육의 경우) 또는 입양자 공제\n\n3. 인적공제: 기본공제와 추가공제를 합친 금액\n\n4. 연금보험료공제: 공적연금 관련법에 따른 연금보험료를 납입한 경우, 해당 과세기간의 종합소득금액에서 납입한 연금보험료를 공제\n\n5. 주택담보노후연금 이자비용공제: 주택임대소득이 있는 거주자가 대통령령으로 정하는 요건에 해당하는 주택담보노후연금을 받은 경우, 해당 과세기간에 발생한 이자비용 상당액을 해당 과세기간의 연금소득금액에서 공제\n\n공제금액은 공제대상자의 연령, 소득, 상태 등에 따라 달라질 수 있으며, 공제한도는 종합소득금액, 총급여액 등에 따라 제한될 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 303, 'prompt_tokens': 3747, 'total_tokens': 4050, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'solar-pro-241126', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fdba6d06-2f02-461e-83fa-1884372c9310-0', usage_metadata={'input_tokens': 3747, 'output_tokens': 303, 'tota

# RetrievalQA 활용하기

In [30]:
from langchain_chroma import Chroma
from langchain_upstage import ChatUpstage
from langchain_upstage import UpstageEmbeddings
from langchain.chains import RetrievalQA

UPSTAGE_API_KEY = 'up_NZqIrRnR6XRpI93EZKtO3UlJhtMWI'
llm = ChatUpstage(model = 'solar-pro',
                  api_key = UPSTAGE_API_KEY)
embeddings = UpstageEmbeddings(model = 'embedding-query',
                               api_key = UPSTAGE_API_KEY)
load_vectorstore = Chroma(collection_name='tax',
                         embedding_function=embeddings,
                         persist_directory='./vectorDB')

retriever = load_vectorstore.as_retriever()

qa_retriever = RetrievalQA.from_llm(
    llm = llm,
    retriever = retriever,
    return_source_documents = True # 응답에 원본 문서 포함 여부
  )

query = '종합소득세 공제 내역'
result = qa_retriever.invoke(query)
print(result['result'])

종합소득세 공제 내역은 다음과 같습니다:

1. 인적공제: 기본공제와 추가공제를 포함합니다. 기본공제는 거주자 본인, 배우자, 부양가족에 대해 1명당 연 150만원을 공제합니다. 추가공제는 경로우대자, 장애인, 부양가족이 있는 여성 세대주, 기본공제대상자인 직계비속 또는 입양자가 있는 경우 연 100만원 등을 추가로 공제합니다.

2. 연금보험료공제: 공적연금 관련법에 따른 기여금 또는 개인부담금을 납입한 경우, 해당 과세기간에 납입한 연금보험료를 종합소득금액에서 공제합니다.

3. 주택담보노후연금 이자비용공제: 연금소득이 있는 거주자가 대통령령으로 정하는 요건에 해당하는 주택담보노후연금을 받은 경우, 해당 과세기간에 발생한 이자비용 상당액을 연금소득금액에서 공제합니다.

4. 특별소득공제: 근로소득이 있는 거주자가 주택자금, 장기주식형저축 등에 대해 공제를 받을 수 있습니다.

5. 자녀세액공제: 기본공제대상자인 자녀에 대해 1명당 15만원(둘째부터는 30만원)을 세액공제합니다.

6. 연금계좌세액공제: 연금계좌에 납입한 금액에 대해 12%(또는 15%)의 세액공제를 받을 수 있습니다.

7. 특별세액공제: 보험료, 의료비, 교육비, 기부금 등에 대해 일정 금액을 세액공제합니다.

이러한 공제 내역을 종합적으로 고려하여 종합소득세 신고를 하면 됩니다.
